In [1]:
#读取数据
import pandas as pd
import torch
import torch.nn as nn
import numpy as np


train_data = pd.read_csv("digit-recognizer-data/train.csv")

'''长教训了QAQ，下面这种写法是错误的
trainX=train_data.values[:][1:]
trainY=train_data.values[:][0]
'''
trainX=train_data.values[:,1:]
trainY=train_data.values[:,0]
#print(trainX.shape)
#print(trainY.shape)

test_data = pd.read_csv("digit-recognizer-data/test.csv")
testX=test_data.values[:,1:]
testY=test_data.values[:,0]

#print(train_data.columns)#输出表头信息

def printImage(id): #可以打印成图像输出
    print('数字为 : ' + str( trainY[id] ))
    for i in range(28):
        for j in range(28):
            print('#' if trainX[id][i*28+j] >0 else '.' ,end="")
        print()

#printImage(0)



c:\Users\24508\Anaconda3\envs\jqxx\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class digitRecognizerModule(nn.Module):#神经网络模型
    def __init__(self):
        super().__init__()
        # 输入尺寸1×28×28
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16,  # 输入通道1 输出通道16
                               kernel_size=(3, 3),  # 卷积核尺寸
                               stride=(1, 1),  # 卷积核每次移动多少个像素
                               padding=1)  # 原图片边缘加几个空白像素
        # conv1输出尺寸16×28×28
        self.pool1 = nn.MaxPool2d(kernel_size=2)  #第一次池化，输出尺寸16×14×14
        self.conv2 = nn.Conv2d(16, 32, 3, 1, 1)  #第二次卷积，输出尺寸32×14×14
        self.pool2 = nn.MaxPool2d(2)  #第二次池化，输出尺寸32×7×7
        self.zhankai = nn.Flatten()#展平为张量，输出尺寸是一维1568(32*7*7)
        self.lin1 = nn.Linear(32 * 7 * 7, 16)#输出尺寸16
        self.jihuo = nn.ReLU()#激活函数
        self.lin2 = nn.Linear(16, 10)#输出尺寸10
    def forward(self,x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.zhankai(x)
        x = self.lin1(x)
        x = self.jihuo(x)
        x = self.lin2(x)
        return x

In [ ]:
net = digitRecognizerModule()
calcLoss = nn.CrossEntropyLoss() #loss用交叉熵来计算
optimizer = torch.optim.SGD(net.parameters(), lr=0.2) #optimizer是优化器，这里使用SGD随机梯度下降。lr是学习率

epochs = 10 #迭代次数
batch_size  = 128 #每批数据的大小为128，那么就有len(trainX)//barch批，每批数据的范围是[i * batch，(i+1) * batch)

# 记录训练过程
train_losses = [] #每个batch的损失
epoch_losses = [] #每个epoch的平均损失
train_accuracies = [] #每个epoch的准确率

for epoch in range(epochs):
    epoch_loss = 0.0
    correct = 0
    total = 0
    for i in range(len(trainX)//batch_size ):
        L = i * batch_size  
        R = (i+1) * batch_size 
        tmpX = trainX[L:R] / 255 #取出来这一批的数据并转为0~1的深度
        tmpX = tmpX.reshape(batch_size , 1, 28, 28) #转化形式，变成能输入网络的形式
        tmpX = torch.from_numpy(tmpX).float() #将numpy转为PyTorch张量

        tmpY = trainY[L:R] #Y同理
        tmpY = torch.from_numpy(tmpY)

        outputs = net(tmpX) #得到网络输出

        
        optimizer.zero_grad() #清空模型参数梯度
        loss = calcLoss(outputs, tmpY)
        loss.backward() #计算损失函数对模型参数的梯度
        optimizer.step() #更新模型参数


        #计算准确率
        _, predicted = torch.max(outputs.data, 1) #沿着张量的第1维计算最大值，返回最大值以及索引
        total += tmpY.size(0)
        correct += (predicted == tmpY).sum().item()
        #记录当前batch损失
        current_loss = loss.item()
        train_losses.append(current_loss)
        epoch_loss += current_loss
    
    #计算epoch指标
    epoch_losses.append(epoch_loss / (len(trainX) // batch_size))
    train_accuracies.append(100 * correct / total)
    
    print(f"Epoch [{epoch+1}/{epochs}] | "
          f"Loss: {epoch_loss / (len(trainX) // batch_size):.4f} | "
          f"Accuracy: {100 * correct / total:.2f}%")



Epoch [1/10] | Loss: 0.5695 | Accuracy: 80.91%
Epoch [2/10] | Loss: 0.0997 | Accuracy: 96.94%
Epoch [3/10] | Loss: 0.0676 | Accuracy: 97.89%
Epoch [4/10] | Loss: 0.0527 | Accuracy: 98.42%
Epoch [5/10] | Loss: 0.0438 | Accuracy: 98.69%
Epoch [6/10] | Loss: 0.0373 | Accuracy: 98.94%
Epoch [7/10] | Loss: 0.0320 | Accuracy: 99.11%
Epoch [8/10] | Loss: 0.0277 | Accuracy: 99.26%
Epoch [9/10] | Loss: 0.0242 | Accuracy: 99.33%
Epoch [10/10] | Loss: 0.0209 | Accuracy: 99.44%
